In [1]:
# Pkg.add("DistributedArrays")
# Pkg.add("BenchmarkTools")

In [1]:
using BenchmarkTools, DistributedArrays

In [2]:
; less .machines.jl

function add_workers(nproc)
    mfile = "/home/juser/.machines"
    mlines = split(readstring(mfile))
    mlines = mlines[1:min(nproc, length(mlines))]
    addprocs(mlines; sshflags="-o StrictHostKeyChecking=no", exename="julia")
end


In [13]:
; less ~/.machines

juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15
juser@10.100.40.15

In [17]:
rmprocs(workers())

numprocs = 3
# try
#     numprocs = parse(Int, ARGS[1])
# end

include(".machines.jl")
add_workers(numprocs)

display(procs())
println("Using $numprocs workers")

@everywhere using DistributedArrays
@everywhere using BenchmarkTools

import Base.run

function run(N)
    
    a = rand(N)
    da = distribute(a)
    da = drand(N)

    #serial_bench = @benchmark sum($a)
    #parallel_bench = @benchmark sum($da)
    
    serial_median   = @elapsed sum(a)
    parallel_median = @elapsed sum(da)

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)

    return (serial_median, parallel_median, 
                serial_median / parallel_median)
#    return parallel_median
end

function run_all()
    Ns = Int[]
    all_data = []

    
    N = 2^20
    
    while N < 10^8
        data = run(N)

        push!(Ns, N)
        push!(all_data, data)

        println("N:", N, "; serial:", data[1],
        "; parallel:", data[2], "; ratio:", data[3])

        N *= 2
    end

    return (Ns, all_data)
end

Ns, all_data = run_all()

11-element Array{Int64,1}:
  1
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14

Using 16 workers


N:1048576; serial:0.000597686; parallel:0.306071148; ratio:0.0019527681844745458
N:2097152; serial:0.001331303; parallel:0.000918487; ratio:1.4494521969282093
N:4194304; serial:0.002871671; parallel:0.001419987; ratio:2.0223220353425773
N:8388608; serial:0.005508586; parallel:0.00229213; ratio:2.4032607225593665
N:16777216; serial:0.011351432; parallel:0.003870686; ratio:2.9326667159258073
N:33554432; serial:0.022333273; parallel:0.006160412; ratio:3.6252888605502362
N:67108864; serial:0.044855413; parallel:0.011226387; ratio:3.9955341821015073


([1048576,2097152,4194304,8388608,16777216,33554432,67108864],Any[(0.000597686,0.306071,0.00195277),(0.0013313,0.000918487,1.44945),(0.00287167,0.00141999,2.02232),(0.00550859,0.00229213,2.40326),(0.0113514,0.00387069,2.93267),(0.0223333,0.00616041,3.62529),(0.0448554,0.0112264,3.99553)])

In [23]:
run(10^7)

(0.006689697,0.003215477,2.0804679989936172)

In [15]:
function run2(N)
    
    # a = rand(N)
    # da = distribute(a)
    da = drand(N)

    #serial_bench = @benchmark sum($a)
    #parallel_bench = @benchmark sum($da)
    
    #serial_median   = @elapsed sum(a)
    bench = @benchmark sum($da)
    
    parallel_median = minimum(bench.times) / N

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)
    
    #return (serial_median, parallel_median, 
     #           serial_median / parallel_median)
#    return parallel_median
end


run2 (generic function with 1 method)

In [16]:
[run2(2^exponent) for exponent in 20:23]

4-element Array{Float64,1}:
 0.307297
 0.217013
 0.200168
 0.247958

In [40]:
function runprod(N)
    
    # a = rand(N)
    # da = distribute(a)
    a = [rand(100,100) for i in 1:N]
    da = distribute(a)

    serial_median = @elapsed prod(a)
    parallel_median = @elapsed prod(da)
    
    #serial_median   = @elapsed sum(a)
    #bench = @elapsed prod(da)
    
    #parallel_median = minimum(bench.times) / N
    #parallel_median = bench / N

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)
    
    return Dict("serial"=>serial_median, "parallel"=>parallel_median, 
    "s/p" => serial_median / parallel_median)
#    return parallel_median
end


runprod (generic function with 1 method)

In [41]:
runprod(10)

Dict{String,Float64} with 3 entries:
  "parallel" => 0.00103884
  "s/p"      => 0.725752
  "serial"   => 0.000753939

In [34]:
rmprocs(workers())

numprocs = 3
# try
#     numprocs = parse(Int, ARGS[1])
# end

include(".machines.jl")
add_workers(numprocs)




3-element Array{Int64,1}:
 15
 16
 17

In [35]:
workers()

3-element Array{Int64,1}:
 15
 16
 17

In [36]:
runprod(10)

LoadError: On worker 15:
UndefVarError: DistributedArrays not defined
 in deserialize at ./serialize.jl:602
 in handle_deserialize at ./serialize.jl:581
 in deserialize at ./serialize.jl:541
 in deserialize_datatype at ./serialize.jl:822
 in handle_deserialize at ./serialize.jl:571
 in deserialize_msg at ./multi.jl:120
 in message_handler_loop at ./multi.jl:1317
 in process_tcp_streams at ./multi.jl:1276
 in #618 at ./event.jl:68
 in #remotecall_fetch#606(::Array{Any,1}, ::Function, ::Function, ::Base.Worker, ::Function, ::Vararg{Any,N}) at ./multi.jl:1070
 in remotecall_fetch(::Function, ::Base.Worker, ::Function, ::Vararg{Any,N}) at ./multi.jl:1062
 in #remotecall_fetch#609(::Array{Any,1}, ::Function, ::Function, ::Int64, ::Function, ::Vararg{Any,N}) at ./multi.jl:1080
 in remotecall_fetch(::Function, ::Int64, ::Function, ::Vararg{Any,N}) at ./multi.jl:1080
 in macro expansion at /home/juser/.julia/v0.5/DistributedArrays/src/core.jl:89 [inlined]
 in (::DistributedArrays.##2#4{Tuple{I

In [37]:
function setup_my_machines_please(N)
    rmprocs(workers())

    numprocs = N
    # try
    #     numprocs = parse(Int, ARGS[1])
    # end

    include(".machines.jl")
    add_workers(numprocs)

    display(procs())
    println("Using $numprocs workers")

    @everywhere using DistributedArrays
    @everywhere using BenchmarkTools
end

setup_my_machines_please (generic function with 1 method)

In [38]:
setup_my_machines_please(3)

4-element Array{Int64,1}:
  1
 18
 19
 20

Using 3 workers


In [42]:
runprod(10)

Dict{String,Float64} with 3 entries:
  "parallel" => 0.00104227
  "s/p"      => 0.8285
  "serial"   => 0.000863522

In [ ]:
runprod(10)

In [43]:
function runprod(matrix_size, N)
    
    # a = rand(N)
    # da = distribute(a)
    a = [rand(matrix_size, matrix_size) for i in 1:N]
    da = distribute(a)

    serial_median = @elapsed prod(a)
    parallel_median = @elapsed prod(da)
    
    #serial_median   = @elapsed sum(a)
    #bench = @elapsed prod(da)
    
    #parallel_median = minimum(bench.times) / N
    #parallel_median = bench / N

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)
    
    return Dict("serial"=>serial_median, "parallel"=>parallel_median, 
    "s/p" => serial_median / parallel_median)
#    return parallel_median
end


runprod (generic function with 2 methods)

In [44]:
runprod(100, 10)

Dict{String,Float64} with 3 entries:
  "parallel" => 0.00127033
  "s/p"      => 0.583753
  "serial"   => 0.000741557

In [45]:
runprod(200, 10)

Dict{String,Float64} with 3 entries:
  "parallel" => 0.00435397
  "s/p"      => 1.27222
  "serial"   => 0.00553919

In [46]:
runprod(400, 10)

Dict{String,Float64} with 3 entries:
  "parallel" => 0.0242881
  "s/p"      => 2.05108
  "serial"   => 0.0498168

In [49]:
runprod(800, 100)

Dict{String,Float64} with 3 entries:
  "parallel" => 1.07692
  "s/p"      => 2.58291
  "serial"   => 2.78159

In [50]:
runprod(1600, 100)

Dict{String,Float64} with 3 entries:
  "parallel" => 7.47475
  "s/p"      => 2.83346
  "serial"   => 21.1794

In [51]:
setup_my_machines_please(4)

5-element Array{Int64,1}:
  1
 21
 22
 23
 24

Using 4 workers


In [52]:
runprod(800, 100)

Dict{String,Float64} with 3 entries:
  "parallel" => 1.66087
  "s/p"      => 1.73844
  "serial"   => 2.88732

In [ ]:
runprod(1600, 100)

In [2]:
using DistributedArrays

In [ ]:
@DArray

In [16]:
function runprod_parallel(matrix_size, N)
    
    # a = rand(N)
    # da = distribute(a)
    #a = [rand(matrix_size, matrix_size) for i in 1:N]
    da = @DArray [rand(matrix_size, matrix_size) for i in 1:N]

    #serial_median = @elapsed prod(a)
    parallel_median = @elapsed prod(da)
    
    #serial_median   = @elapsed sum(a)
    #bench = @elapsed prod(da)
    
    #parallel_median = minimum(bench.times) / N
    #parallel_median = bench / N

    #serial_median = median(serial_bench.times)
    #parallel_median = median(parallel_bench.times)
    
    return Dict("matrix_size"=>matrix_size, "N"=>N, "parallel"=>parallel_median)
#    return parallel_median
end


runprod_parallel (generic function with 1 method)

In [ ]:
data_dicts = [runprod_parallel(matrix_size, 10) for matrix_size in 2.^(5:12)]

In [1]:
for data in data_dicts
    matrix_size = data["matrix_size"]
    time = data["parallel"]
    N = data["N"]
    
    println(matrix_size, "\t", time, "\t", time/(2matrix_size^3 * N))
end

LoadError: UndefVarError: data_dicts not defined

## Dictionaries

"hash table" / "associative array", contructed out of "pairs"

In [3]:
p = 3 => "hello"

3=>"hello"

In [4]:
typeof(p)

Pair{Int64,String}

In [5]:
d = Dict(3 => "hello", 5=>"goodbye")

Dict{Int64,String} with 2 entries:
  3 => "hello"
  5 => "goodbye"

In [6]:
d[3]

"hello"

In [7]:
d[5]

"goodbye"

In [8]:
v = ["", "", "hello", "", "goodbye"]

5-element Array{String,1}:
 ""       
 ""       
 "hello"  
 ""       
 "goodbye"

In [9]:
v[3]

"hello"

In [10]:
v[5]

"goodbye"

In [11]:
using Interact

INFO: Precompiling module DataStructures.


In [14]:
@manipulate for key in d
    key
end

Interact.Options{:ToggleButtons,String}(Signal{String}(hello, nactions=1),"key","hello","3",Interact.OptionDict(DataStructures.OrderedDict("3"=>"hello","5"=>"goodbye"),Dict("goodbye"=>"5","hello"=>"3")),Any[],Any[],true,"horizontal")

"hello"

A dictionary has *keys* and *values*:

In [15]:
keys(d)

Base.KeyIterator for a Dict{Int64,String} with 2 entries. Keys:
  3
  5

In [16]:
values(d)

Base.ValueIterator for a Dict{Int64,String} with 2 entries. Values:
  "hello"
  "goodbye"

In [17]:
collect(keys(d))  # make it into an array

2-element Array{Int64,1}:
 3
 5

In [18]:
d

Dict{Int64,String} with 2 entries:
  3 => "hello"
  5 => "goodbye"

In [19]:
d[2]

LoadError: KeyError: key 2 not found

In [20]:
d[2] = "new"

"new"

In [21]:
d

Dict{Int64,String} with 3 entries:
  2 => "new"
  3 => "hello"
  5 => "goodbye"

In [22]:
@manipulate for key in d
    key
end

Interact.Options{:ToggleButtons,String}(Signal{String}(new, nactions=1),"key","new","2",Interact.OptionDict(DataStructures.OrderedDict("2"=>"new","3"=>"hello","5"=>"goodbye"),Dict("new"=>"2","goodbye"=>"5","hello"=>"3")),Any[],Any[],true,"horizontal")

"new"

In [23]:
d[-1] = "negative"

"negative"

In [24]:
d

Dict{Int64,String} with 4 entries:
  2  => "new"
  3  => "hello"
  -1 => "negative"
  5  => "goodbye"

In [25]:
d[3.1] = "float"

LoadError: InexactError()

`InexactError` means that 3.1 cannot be "fitted in" to an `Int`.

In [26]:
d[7.0] = "not sure if float or not"

"not sure if float or not"

In [27]:
d

Dict{Int64,String} with 5 entries:
  7  => "not sure if float or not"
  2  => "new"
  3  => "hello"
  -1 => "negative"
  5  => "goodbye"

In [30]:
any_dict = Dict(3=>"hello", "goodbye"=>"whatever")

Dict{Any,String} with 2 entries:
  "goodbye" => "whatever"
  3         => "hello"

In [33]:
d = Dict()  # constructor of an empty dictionary

Dict{Any,Any} with 0 entries

In [35]:
d["hello"] = 3

3

In [36]:
d

Dict{Any,Any} with 1 entry:
  "hello" => 3

Construct an empty dictionary with given type parameters, e.g. `Float64` and `String`:

In [38]:
Dict{Float64, String}  # the name of the type

Dict{Float64,String}

In [40]:
d = Dict{Float64, String}()  # constructs an object of that type

Dict{Float64,String} with 0 entries

In [42]:
push!(d, 3.1=>"hello")

Dict{Float64,String} with 1 entry:
  3.1 => "hello"

In [43]:
d[3.1] = "hello"

"hello"

Can a key have multiple values? No, but: [DataStructures.jl](https://github.com/JuliaLang/DataStructures.jl/).

Use a dict as a multi-dict:

In [45]:
d = Dict("hello" => [3, 4], "goodbye" => Int[])

Dict{String,Array{Int64,1}} with 2 entries:
  "goodbye" => Int64[]
  "hello"   => [3,4]

In [47]:
d["hello"]

2-element Array{Int64,1}:
 3
 4

In [48]:
push!(d["hello"], 7, 3)

4-element Array{Int64,1}:
 3
 4
 7
 3

In [49]:
d

Dict{String,Array{Int64,1}} with 2 entries:
  "goodbye" => Int64[]
  "hello"   => [3,4,7,3]

In [50]:
d1 = Dict("a"=>1, "b"=>2)
d2 = Dict("a"=>3, "b"=>4)

Dict{String,Int64} with 2 entries:
  "b" => 4
  "a" => 3

In [51]:
d = Dict("first" => d1, "second" => d2)

Dict{String,Dict{String,Int64}} with 2 entries:
  "second" => Dict("b"=>4,"a"=>3)
  "first"  => Dict("b"=>2,"a"=>1)

In [52]:
d["second"]

Dict{String,Int64} with 2 entries:
  "b" => 4
  "a" => 3

In [54]:
d["third"] = Dict()

Dict{Any,Any} with 0 entries

In [55]:
d

Dict{String,Dict{String,Int64}} with 3 entries:
  "second" => Dict("b"=>4,"a"=>3)
  "first"  => Dict("b"=>2,"a"=>1)
  "third"  => Dict{String,Int64}()

In [56]:
d["fourth"] = Dict(3.1 => 17)

LoadError: MethodError: Cannot `convert` an object of type Float64 to an object of type String
This may have arisen from a call to the constructor String(...),
since type constructors fall back to convert methods.

In [61]:
@manipulate for key in d
    key
end

Interact.Options{:ToggleButtons,Dict{String,Int64}}(Signal{Dict{String,Int64}}(Dict("b"=>4,"a"=>3), nactions=1),"key",Dict("b"=>4,"a"=>3),"second",Interact.OptionDict(DataStructures.OrderedDict("second"=>Dict("b"=>4,"a"=>3),"first"=>Dict("b"=>2,"a"=>1),"third"=>Dict{String,Int64}()),Dict(Dict("b"=>2,"a"=>1)=>"first",Dict{String,Int64}()=>"third",Dict("b"=>4,"a"=>3)=>"second")),Any[],Any[],true,"horizontal")

Dict{String,Int64} with 2 entries:
  "b" => 4
  "a" => 3

In [62]:
d

Dict{String,Dict{String,Int64}} with 3 entries:
  "second" => Dict("b"=>4,"a"=>3)
  "first"  => Dict("b"=>2,"a"=>1)
  "third"  => Dict{String,Int64}()

In [67]:
d["first"]["a"]  

1

In [66]:
d["first", "a"]  # not this

LoadError: KeyError: key ("first","a") not found

In [68]:
hash([1,2,3])

0xd468fb40d24a17cf

In [69]:
d = Dict([1,2,3]=>"hello")

Dict{Array{Int64,1},String} with 1 entry:
  [1,2,3] => "hello"

In [70]:
d[[1,2,3]]

"hello"

In [71]:
d

Dict{Array{Int64,1},String} with 1 entry:
  [1,2,3] => "hello"

In [72]:
v = [Dict(i => string(i)) for i in 1:10 ]

10-element Array{Dict{Int64,String},1}:
 Dict(1=>"1")  
 Dict(2=>"2")  
 Dict(3=>"3")  
 Dict(4=>"4")  
 Dict(5=>"5")  
 Dict(6=>"6")  
 Dict(7=>"7")  
 Dict(8=>"8")  
 Dict(9=>"9")  
 Dict(10=>"10")

In [73]:
using DistributedArrays

In [74]:
dv = distribute(v)

10-element DistributedArrays.DArray{Dict{Int64,String},1,Array{Dict{Int64,String},1}}:
 Dict(1=>"1")  
 Dict(2=>"2")  
 Dict(3=>"3")  
 Dict(4=>"4")  
 Dict(5=>"5")  
 Dict(6=>"6")  
 Dict(7=>"7")  
 Dict(8=>"8")  
 Dict(9=>"9")  
 Dict(10=>"10")

In [75]:
summary(dv)

"10-element DistributedArrays.DArray{Dict{Int64,String},1,Array{Dict{Int64,String},1}}"

In [76]:
vec_functions = [x->i*x for i in 1:10]

10-element Array{##44#46{Int64},1}:
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44

In [78]:
vec_functions[1](10)

10

In [79]:
vec_functions[2](10)

20

In [80]:
dfunc = distribute(vec_functions)

10-element DistributedArrays.DArray{##44#46{Int64},1,Array{##44#46{Int64},1}}:
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44
 #44

In [82]:
dfunc[1](10)

10

In [83]:
values = [1:10;]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [84]:
dvalues = distribute(values)

10-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [85]:
map( (f,v)->f(v), dfunc, dvalues )

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100